In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

from keras.optimizers import SGD, Adam, Adadelta
from keras.layers import Dense, Flatten, Input, Activation
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dropout
import keras
from keras import regularizers

import h5py  # compress and save features

E:\Anaconda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('salary_train_fixed.csv')
train.head()

,Category,ContractTime,ContractType,FullDescription,Id,SalaryNormalized,SalaryRaw,company,location_1,location_2,location_3,location_4,location_5,location_6,sourcename,title
0,8,1,0,Engineering Systems Analyst Dorking Surrey Sal...,12612628,25000,20000 - 30000/annum 20-30K,912,1,0,53,201,353,462,42,0
1,8,1,0,Stress Engineer Glasgow Salary **** to **** We...,12612830,30000,25000 - 35000/annum 25-35K,912,1,0,0,0,962,29,42,0
2,8,1,0,Mathematical Modeller / Simulation Analyst / O...,12612844,30000,20000 - 40000/annum 20-40K,912,1,0,0,0,1004,23,42,0
3,8,1,0,Engineering Systems Analyst / Mathematical Mod...,12613049,27500,25000 - 30000/annum 25K-30K negotiable,912,1,0,0,185,1066,11,42,0
4,8,1,0,"Pioneer, Miser Engineering Systems Analyst Do...",12613647,25000,20000 - 30000/annum 20-30K,912,1,0,0,185,1066,11,42,0


In [ ]:
test = pd.read_csv('salary_test_fixed.csv')
test.head()

In [3]:
Category = train['Category']

ContractTime = train['ContractTime']

ContractType = train['ContractType']

location_1 = train['location_1']

location_2 = train['location_2']

location_3 = train['location_3']

location_4 = train['location_4']

location_5 = train['location_5']

location_6 = train['location_6']

company = train['company']

sourcename = train['sourcename']

title = train['title']

In [4]:
location_1_onehot = keras.utils.to_categorical(location_1.values, len(set(location_1))) 
location_1_onehot.shape

(244768, 2)

In [5]:
location_2_onehot = keras.utils.to_categorical(location_2.values, len(set(location_2))) 
location_2_onehot.shape

(244768, 12)

In [6]:
location_3_onehot = keras.utils.to_categorical(location_3.values, 74) 
location_3_onehot.shape

(244768, 74)

In [7]:
location_4_onehot = keras.utils.to_categorical(location_4.values, 241) 
location_4_onehot.shape

(244768, 241)

In [8]:
location_5_onehot = keras.utils.to_categorical(location_5.values, 1239) 
location_5_onehot.shape

(244768, 1239)

In [9]:
location_6_onehot = keras.utils.to_categorical(location_6.values, len(set(location_6))) 
location_6_onehot.shape

(244768, 2088)

In [ ]:
company_onehot = keras.utils.to_categorical(company, len(set(company))) 
company_onehot.shape

In [10]:
features = np.concatenate([location_1_onehot, location_2_onehot, location_3_onehot,
                          location_4_onehot, location_5_onehot, location_6_onehot], axis=-1) # with F

In [11]:
kk = features.astype(int)

In [12]:
kk.dtype

dtype('int32')

In [13]:
kk.shape

(244768, 3656)

In [14]:
from sklearn.decomposition import PCA 

In [17]:
pca = PCA(n_components=1000)  

In [20]:
newX=pca.fit_transform(kk)

KeyboardInterrupt: 

In [21]:
newX

NameError: name 'newX' is not defined

In [ ]:
kk = company_onehot[0:10]

In [ ]:
kk.shape

In [ ]:
float(kk)

In [ ]:
type(kk)

In [ ]:
kk = np.concatenate(int(company_onehot[0]),company_onehot[1])

In [ ]:
kk.shape

In [ ]:
X.shape

In [ ]:
bb = pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_) 

In [ ]:
X

In [ ]:
features_train = np.vstack([title, location_1, location_2, location_3,
                          location_4, location_5, location_6, ContractType,
                          ContractTime, company, Category, sourcename])
features_train = np.transpose(features_train)
features_train.shape

In [ ]:
Category = test['Category']

ContractTime = test['ContractTime']

ContractType = test['ContractType']

location_1 = test['location_1']

location_2 = test['location_2']

location_3 = test['location_3']

location_4 = test['location_4']

location_5 = test['location_5']

location_6 = test['location_6']

company = test['company']

sourcename = test['sourcename']

title = test['title']

In [ ]:
features_test = np.vstack([title, location_1, location_2, location_3,
                          location_4, location_5, location_6, ContractType,
                          ContractTime, company, Category, sourcename])
features_test = np.transpose(features_test)
features_test.shape

In [ ]:
salay_train = train['SalaryNormalized']
salay_test = test['SalaryNormalized']

In [ ]:
s1 = np.log(salay_train)
s2 = np.log(salay_test)

# build model

In [ ]:
# create model
model = Sequential()
model.add(Dense(24,input_dim=12, init='uniform'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))


model.add(Dense(1))


# Compile model
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer='sgd') ####potential error
print(model.summary())

In [ ]:
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=40, verbose=0, mode='min')
# Change log_dir for differnt activation function, change the number for every run.
tbCallBack = keras.callbacks.TensorBoard(log_dir='./tensorboard/11', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
hist = model.fit(features_train, s1, batch_size=1024, epochs=200, shuffle=True,verbose=2,validation_split=0.05, callbacks = [tbCallBack,early_stopping_monitor])